In [1]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD


## Analyse Dataset


### Steam Video Games
https://www.kaggle.com/datasets/trolukovich/steam-games-complete-dataset

In [2]:
import csv

file = open('steam_games.csv',encoding="utf8")
csvreader = csv.reader(file)

In [3]:
header = []
header = next(csvreader)
header

['url',
 'types',
 'name',
 'desc_snippet',
 'recent_reviews',
 'all_reviews',
 'release_date',
 'developer',
 'publisher',
 'popular_tags',
 'game_details',
 'languages',
 'achievements',
 'genre',
 'game_description',
 'mature_content',
 'minimum_requirements',
 'recommended_requirements',
 'original_price',
 'discount_price']

In [4]:
len(header)

20

In [5]:
rows = []
for row in csvreader:
        rows.append(row)


In [6]:
file.close()

In [7]:
len(rows)


40833

In [8]:
types=[]
for i in range(len(rows)):
    types.append(rows[i][1])
name=[]
for i in range(len(rows)):
    name.append(rows[i][2])
    
    
developer=[]
for i in range(len(rows)):
    developer.append(rows[i][7]) 
    
publisher=[]
for i in range(len(rows)):
    publisher.append(rows[i][8])   
    
    
popular_tags=[]
for i in range(len(rows)):
    popular_tags.append(rows[i][9])   
    
game_details=[]
for i in range(len(rows)):
    game_details.append(rows[i][10])        


minimum_requirements=[]
for i in range(len(rows)):
    minimum_requirements.append(rows[i][16])  
    

    
game_description=[]
for i in range(len(rows)):
    game_description.append(rows[i][14])  
    
genre=[]
for i in range(len(rows)):
    genre.append(rows[i][13])   


In [9]:
df  = pd.DataFrame()

In [10]:
df['name']=name
df['genre']=genre
df['game_description']=game_description
df['game_details']=game_details
df['minimum_requirements']=minimum_requirements

df['popular_tags']=popular_tags
df['publisher']=publisher
df['developer']=developer
df['types']=types

### Create New column to store important tags
- Will use this column to find similarity among games

In [11]:
df.shape

(40833, 9)

In [12]:
#First get all tokens from overview columns
df['genre'] = df['genre'].apply(lambda x:x.split())

In [13]:
df['game_details'] = df['game_details'].apply(lambda x:x.split())

In [14]:
df['popular_tags'] = df['popular_tags'].apply(lambda x:x.split())

In [15]:
df['developer']=df['developer'].replace(' ', '_', regex=True)

In [16]:
df['developer']=df['developer'].apply(lambda x:x.split())

In [17]:

df['tag'] = df['game_details'] + df['popular_tags']+df['developer']


In [18]:
df['tag'] =df['tag'] .apply(lambda x: " ".join(x))

## Final dataset

In [19]:
df.head(2)

,name,genre,game_description,game_details,minimum_requirements,popular_tags,publisher,developer,types,tag
0,DOOM,[Action],"About This Game Developed by id software, the...","[Single-player,Multi-player,Co-op,Steam, Achie...","Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","[FPS,Gore,Action,Demons,Shooter,First-Person,G...","Bethesda Softworks,Bethesda Softworks",[id_Software],app,"Single-player,Multi-player,Co-op,Steam Achieve..."
1,PLAYERUNKNOWN'S BATTLEGROUNDS,"[Action,Adventure,Massively, Multiplayer]",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,"[Multi-player,Online, Multi-Player,Stats]","Minimum:,Requires a 64-bit processor and opera...","[Survival,Shooter,Multiplayer,Battle, Royale,P...","PUBG Corporation,PUBG Corporation",[PUBG_Corporation],app,"Multi-player,Online Multi-Player,Stats Surviva..."


### Contant based Filtering

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vector = cv.fit_transform(df['tag']).toarray()


In [21]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity = cosine_similarity(vector)

### Recommendations

In [22]:
df[df['name'] == 'DOOM'].index[0]

0

In [51]:
def Recommendgames(game,df,similarity):
    index  = df[df['name'] == game].index [0]
    distance = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    result=[]
    for i in distance[1:10]:
        result.append(df.iloc[i[0]][0])
    return result    

In [52]:
resuls=Recommendgames('DOOM',df,cosine_similarity)

In [53]:
result_df=df[df["name"].isin(resuls)]
result_df

,name,genre,game_description,game_details,minimum_requirements,popular_tags,publisher,developer,types,tag
139,DUSK,"[Action,Indie]",About This Game DUSK reintroduces you to a w...,"[Single-player,Online, Multi-Player,Steam, Ach...","Minimum:,OS:,Windows 7 Or Later,Processor:,2.4...","[FPS,Retro,Action,Fast-Paced,Great, Soundtrack...","New Blood Interactive,New Blood Interactive",[David_Szymanski],app,"Single-player,Online Multi-Player,Steam Achiev..."
183,Call of Duty®: Black Ops II,[Action],About This Game Pushing the boundaries of wha...,"[Single-player,Multi-player,Co-op,Steam, Achie...",,"[Action,Multiplayer,FPS,Shooter,First-Person,S...","Activision,Activision",[Treyarch],app,"Single-player,Multi-player,Co-op,Steam Achieve..."
1294,Painkiller Hell & Damnation,[Action],About This Game Heaven's Hitman is Back... W...,"[Single-player,Multi-player,Co-op,Shared/Split...","Minimum:,OS:,Windows XP SP3, Windows Vista/7,P...","[Action,FPS,Gore,Co-op,Horror,Singleplayer,Sho...","THQ Nordic,THQ Nordic",[The_Farm_51],app,"Single-player,Multi-player,Co-op,Shared/Split ..."
1458,The Typing of The Dead: Overkill,[Action],About This Game Typing of the dead: Overkill ...,"[Single-player,Multi-player,Steam, Achievement...","Minimum:,OS:,Windows 7 / Windows 8,Processor:,...","[Typing,On-Rails, Shooter,Zombies,Comedy,Actio...","SEGA,SEGA",[Modern_Dream],app,"Single-player,Multi-player,Steam Achievements,..."
2283,F.E.A.R. 3,[Action],About This Game Alma is expecting and a new l...,"[Single-player,Multi-player,Co-op,Steam, Achie...","Minimum:,OS:,Windows XP,Processor:,Intel Core ...","[FPS,Horror,Co-op,Action,Multiplayer,Shooter,S...","Warner Bros. Interactive Entertainment,Warner ...",[Day_1_Studios],app,"Single-player,Multi-player,Co-op,Steam Achieve..."
2348,Serious Sam HD: The First Encounter,"[Action,Indie]",About This Game The classic arcade FPS Seriou...,"[Single-player,Multi-player,Co-op,Steam, Achie...",,"[FPS,Action,Co-op,Shooter,Comedy,Multiplayer,G...","Devolver Digital,Devolver Digital",[Croteam],app,"Single-player,Multi-player,Co-op,Steam Achieve..."
2432,Serious Sam HD: The Second Encounter,"[Action,Indie]",About This Game The glorious high-definition ...,"[Single-player,Multi-player,Co-op,Steam, Achie...",,"[FPS,Action,Co-op,Shooter,Comedy,Multiplayer,I...","Devolver Digital,Devolver Digital",[Croteam],app,"Single-player,Multi-player,Co-op,Steam Achieve..."
21123,UNLOVED,[Action],About This Game UNLOVED is based on the commu...,"[Single-player,Multi-player,Co-op,Steam, Achie...","Minimum:,OS:,Windows 10, Windows 8, Windows 7 ...","[Action,Horror,FPS,Gore,Co-op,Multiplayer,Dark...","Paul Schneider,Paul Schneider",[BlueEagle_Productions],app,"Single-player,Multi-player,Co-op,Steam Achieve..."
27648,Rise of the Triad,[Action],About This Game Apogee Software and Intercept...,"[Single-player,Multi-player,Online, Multi-Play...","Minimum:,OS:,Windows XP or Vista 32-bit,Proces...","[FPS,Action,Old, School,Fast-Paced,Remake,Gore...","Apogee Software,Apogee Software",[Slipgate_Ironworks™],app,"Single-player,Multi-player,Online Multi-Player..."
